# Analysis of `vehicle-zone-tracking.xml`

In [ ]:
import pprint, math, datetime
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

# normal = "berlin-1pct"
# normal = "../1pct-output-test"
normal = "normal-output"

scenarios = [
  "apats-output"
#   "greater-berlin-1pct"
]

In [ ]:
def get_distances_per_zone(file_path, freeze_zones=False):
    distances_per_zone = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        current_zone_timestep = element.attrib["zone-timestep"]

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            speed = float(vehicle.attrib["speed"])
            v_timestep = vehicle.attrib["zone-timestep"]

            distances = {}
            if vid in distances_per_zone:
                distances = distances_per_zone[vid]

            for polygon in vehicle.getchildren():
                pid_parts = polygon.attrib["id"].split("_")
                pid = pid_parts[0]
                p_timestep = pid_parts[1]

                if freeze_zones:
                    if p_timestep != v_timestep:
                        continue
                else:
                    if p_timestep != current_zone_timestep:
                        continue

                if pid.startswith("hole"):
                    hole, hole_counter, zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] -= speed
                    else:
                        distances[zone] = -speed
                else:
                    zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] += speed
                    else:
                        distances[zone] = speed
            
            distances_per_zone[vid] = distances

        element.clear()

    return distances_per_zone

n_distances = get_distances_per_zone(f"{normal}/vehicle-zone-tracking.xml")

scenario_data = {}

for s in scenarios:
    distances = get_distances_per_zone(f"{s}/vehicle-zone-tracking.xml")
    scenario_data[s] = distances

print("Done")

### Distances per trip

In [ ]:
pprint.pprint(n_distances)

### Distances per person

In [ ]:
n_distances_pp = {}

for trip in n_distances:
    person, counter = trip.split("_")

    if person in n_distances_pp:
        person_distances = n_distances_pp[person]
        trip_distances = n_distances[trip]

        for zone in trip_distances:
            if zone in person_distances:
                person_distances[zone] += trip_distances[zone]
            else:
                person_distances[zone] = trip_distances[zone]
                
        n_distances_pp[person] = person_distances

    else:
        n_distances_pp[person] = n_distances[trip]

pprint.pprint(n_distances_pp)